In [1]:
import numpy as np
import pandas as pd
import sys
sys.path.append('../')
from entropy_bounds.utils import optimization_cf
from binaryIV import simulate_deterministic_data_with_probabilistic_ate, extract_prob_dict, entropy_of_array



import matplotlib.pyplot as plt
import seaborn as sns
import math


In [5]:
x = .5 # The marginal probability P(x) that corresponding to P(y|do(x))
y_x = .5 # The conditional probability P(y|x) that corresponding to p(y|do(x))
entr = 100 # The upper bound of confounder entropy
x_bar = 1 - x
y_bar_x = 1 - y_x

In [8]:
## Construct the joint distribution. Our experiment shows the bounds only depends on the value of p(y|x) and p(x), we pick arbitrary values for other entries.
pyx = np.array([[y_x*x, y_bar_x*x], [x_bar*0.5 , x_bar*0.5]]).T
lb, ub, _ = optimization_cf(pyx, ub=entr, p =0, q=0)
print(f"The bounds of P(y|do(x)) is [{round(lb,3)}, {round(ub,3)}]")

The bounds of P(y|do(x)) is [0.25, 0.75]


In [98]:
sim = simulate_deterministic_data_with_probabilistic_ate()

df = pd.DataFrame({'Y': sim['Y'], 'X': sim['X'], 'Z': sim['Z']})
prob_dict = extract_prob_dict(df)

# # pYX_Z = P(Y,X|Z)
# print('Probability Dictionary:')
# for key, value in prob_dict.items():
#     print(f"{key}: {value}")

##compute p(y|x) from df
y_x = df[df['X'] == 1]['Y'].mean()

##compute p(x) from df
x = df['X'].mean()

##confounder entropy
entr = 100


x_bar = 1 - x
y_bar_x = 1 - y_x

pyx = np.array([[y_x*x, y_bar_x*x], [x_bar*0.5 , x_bar*0.5]]).T
lb_p11, ub_p11, _ = optimization_cf(pyx, ub=entr, p =0, q=1)

####
##compute p(y|x) from df
y_x = df[df['X'] == 1]['Y'].mean()

##compute p(x) from df
x = df['X'].mean()

x_bar = 1 - x
y_bar_x = 1 - y_x

pyx = np.array([[y_x*x, y_bar_x*x], [x_bar*0.5 , x_bar*0.5]]).T
lb_p10, ub_p10, _ = optimization_cf(pyx, ub=entr, p =0, q=0)

bound_lower = lb_p11 - lb_p10
bound_upper = ub_p11 - ub_p10

bounds_valid = bound_lower <= sim['ATE_true'] <= bound_upper

result = {
    'b_X_Y': sim['b_X_Y'],
    'b_Z': sim['b_Z'],
    'b_U_X': sim['b_U_X'],
    'b_U_Y': sim['b_U_Y'],
    'entropy_Y': entropy_of_array(sim['Y']),
    'entropy_X': entropy_of_array(sim['X']),
    'entropy_Z': entropy_of_array(sim['Z']),
    'entropy_U': entropy_of_array(sim['U']),
    'corr_X_Y': np.corrcoef(sim['X'], sim['Y'])[0, 1],
    'corr_X_Z': np.corrcoef(sim['X'], sim['Z'])[0, 1],
    'corr_Y_Z': np.corrcoef(sim['Y'], sim['Z'])[0, 1],
    'ATE_true': sim['ATE_true'],
    'bound_lower': bound_lower,
    'bound_upper': bound_upper,
    'bound_width': bound_upper - bound_lower,
    'bounds_valid': bounds_valid
}

for key, value in result.items():
    print(f"{key}: {value}")

b_X_Y: -1.095861795318481
b_Z: -1.0675216543109356
b_U_X: -0.7139098495776162
b_U_Y: 1.0949174617189184
entropy_Y: 0.6927551293225422
entropy_X: 0.6253527126087997
entropy_Z: 0.6930751788318458
entropy_U: 0.6924990405193354
corr_X_Y: -0.31557179787439665
corr_X_Z: -0.2537956703596456
corr_Y_Z: 0.01634358426027675
ATE_true: -0.2495122385388485
bound_lower: 0.25100723245233475
bound_upper: -0.11300760398211884
bound_width: -0.3640148364344536
bounds_valid: False
